In [1]:
# library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import re
import glob
import shutil
from pathlib import Path
# showing module
from IPython.display import display

# output display option adjustment
# precision of floating point in numpy
np.set_printoptions(suppress=True, precision=4)

# precision of floating point in pandas
pd.options.display.float_format = '{:.4f}'.format

# display all columns in dataframe
pd.set_option("display.max_columns",None)

# default font size in graph
plt.rcParams["font.size"] = 14

# graph display
sns.set(rc={'figure.figsize':(12,5)});
plt.figure(figsize=(12,5));

# random seed
random_seed = 45

<Figure size 864x360 with 0 Axes>

In [2]:
# # input_dir（input directory） を作ります
current_note_path = os.path.dirname(os.path.abspath('__file__'))
INPUT_DIR = os.path.join(current_note_path, "inputs")

# INPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(INPUT_DIR):
    os.mkdir(INPUT_DIR)

# output_dir(output directory) を作ります
OUTPUT_DIR = os.path.join(current_note_path, 'outputs')

# OUTPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [7]:
# csvファイルを `data` ディレクトリ（=フォルダー） に移動させます
unique_dir_names = []
for f in Path(f'{current_note_path}').rglob('*.csv'):
    unique_dir_names.append(f)

for file in list(set(unique_dir_names)):
    print(f'moved file: {file}')
    shutil.move(f'{file}', f'{INPUT_DIR}')

moved file: /Users/satoshiido/Documents/coding_general/signate/LT/time_series_wide.csv


In [3]:
# csv を読み取る関数を設定したあげると、pathや拡張子を書かずに読み込めるので入力が楽になります
def read_csv(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.csv')
    print(f'Load: {path}')
    # dask等に変更する場合は以下のコマンドを変更する
    return pd.read_csv(path, **kwrgs)

# 10章の構成

> この章では以下の3つを中心に説明します

* 時系列データの種類や性質、注意点、データの操作について
  * 「3.10.1 時系 列データとは?」
  * 「3.10.2 予測する時点より過去の情報のみを使う」
  * 「3.10.3 ワイド フォーマットとロングフォーマット」
* 時系列データから特徴量を作成する方法について
  * 「3.10.4 ラグ特徴量」 
  * 「3.10.5 時点と紐付いた特徴量を作る」
* 分析コンペのデータの形式上、使えるデータの期間についてさらに注意が必要な点
  * 「3.10.6 予測に使えるデータの期間」

# 3.10.1 時系列データとは

1. 時間情報を持つ変数があるかどうか\
   -> <b>時間の情報を上手く使って特徴量を作る</b>
   
2. 学習データ・テストデータが時系列で分かれているか、時間に沿って分割したバリデーションを行う必要があるかどうか\
   -> <b>時間に沿って分割したバリデーションを行うとともに、特徴量についても将来の情報を不適切に使わないように気を付ける</b>

3. ユーザや店舗といった系列ごとに時系列の目的変数があり、「3.10.4 ラグ特徴量」で説明するラグ特徴量がとれる形式であるかどうか\
   -> <b>過去の目的変数が将来の予測に重要な情報となるため、ラグ特徴量を作る</b>

具体的な分析コンペのタスク例を考えてみる

ケースa (上記の１に当てはまる場合)
* ユーザの属性や過去の行動ログが与えられる
* 予測対象は1か月以内に解約するかどうか
* ある時点のユーザを分割して、学習データ・テストデータが作成されている（学習データとテストデータのユーザー情報は同時点のもの）

ケースb (1.と2.に当てはまる場合)
* ユーザの属性や過去の行動ログが与えられる
* 予測対象は1か月以内に解約するかどうか
* テストデータはある時点のユーザ全体で、学習データとして、過去の各月ごとに月初に存在するユーザとその月内に退会したかどうかが与えられる（各ユーザーごとに複数行ある。テストデータで予測する対象のユーザーが学習データにも含まれている）

ケースc (1.と2.と3.に当てはまる場合)
* ユーザの属性や過去の行動ログのほかに、ユーザー過去の利用時間が日ごとに与えられる
* 予測対象は日ごとの利用時間
* テストデータはある時点のユーザ全体と将来の一定期間の各日の組み合わせで、ユーザの過去の利用時間から学習データが作成できる（ワイドフォーマット形式のデータであることも多く、普段使っているロングフォーマットに変換した上でラグ特徴量を作成する必要がある）

# 3.10.2 予測する時点より過去の情報のみを使う

前述のケースbやケースcの場合では、将来のデータを不適切に使うと、目的変数のリークを起こす可能性がある。


* 目的変数が、過去の目的変数の情報を含むことがある
    * 将来に来店数が増えていれば、それまでも来店数が増えている可能性が高い
    * 10年後の平均気温が分かっていれば、8年後の平均気温の予測は容易になる
* 目的変数以外のデータも、過去の目的変数の情報を含むことがある
  * ある月の行動ログがないのなら、それ以前に解約したかもしれない(解約したか否かという目的変数の情報を含む)
* ある商品のプロモーションの増加は、それ以前にその商品の販売が好調だったことが原因かもしれない(商品の販売数という目的変数の情報を含む)

> 目的変数だけでなく、目的変数以外のデータについても時点を意識する必要がある。\
> つまり時系列データをクリーンに扱うには、予測する時点より過去の情報のみを使う制約を守って特徴量の作成やバリデーションを行う必要がある

1. 特徴量作成において、あるレコードの特徴量にはその時点より先のデータを使わないように作成する
2. バリデーションにおいて、学習データにバリデーションデータより将来のレコードを含めないようにする

# 3.10.3 ワイドフォーマットとロングフォーマット

> *キーとなる変数A、Bを行および列として、注目する変数Cを値とする形式のテーブルを本書ではワイドフォーマットと呼ぶ\
> *キーとなる変数A、Bともに列として、注目する変数だけでなく他の変数も列として含むことができる形式のテーブルを本節ではロングフォーマットと呼ぶ（多くのテーブルデータがこの形式）

[両者の違いを簡単に説明しているドキュメント(若干意味合いが違うかも)](https://minnanods.com/wide-vs-long-data/)

<b>ワイドフォーマット</b>

* 注目する変数しか保持できないがその変数の時系列的な変化が見やすく、また後述のラグ特徴量をとる場合に扱いやすい

<b>ロングフォーマット</b>

* 機械学習を行うときに必要な形式。オーソドックスな形式
* *上記のワイドフォーマットデータの場合は日、ユーザごとに目的変数を持つロングフォーマットにする必要がある

時系列データがある場合は
**ロングフォーマット→ワイドフォーマットに変換→特徴量生成→ロングフォーマットに変換→学習**みたいな流れになる

変換の概要
* ワイド→ロング
  * DataFrameのStackメソッドを使用する
* ロング→ワイド
  * DataFrameのpivotメソッドを使用する

[データ変換についてPandasの参考ドキュメント](https://pandas.pydata.org/docs/user_guide/reshaping.html)

MultiIndexが出てきた時の操作方法[（そもそもMultiIndexとは）](https://qiita.com/papi_tokei/items/05fe87195f7a772c567a)
* 行のMultiIndexはDataFrameのreset_indexメソッドでIndexからテーブルの値にする
* 列のMultiIndexは、MultiIndexのto_flat_indexメソッドでタプルを値とするIndexにする

In [4]:
# ワイドフォーマットのデータを読み込む
df_wide = read_csv('time_series_wide', index_col=0)

# インデックスの型を日付型に変更する
df_wide.index = pd.to_datetime(df_wide.index)

df_wide.iloc[:5, ]

Load: /Users/satoshiido/Documents/coding_general/signate/LT#1/inputs/time_series_wide.csv


,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711
2016-07-04,937,5451,1977
2016-07-05,881,4729,1975


In [5]:
# ロングフォーマットに変換する（以下のreset_indexの部分をいじると違いがわかってくる）
df_long = df_wide.stack().reset_index(1)
df_long.columns = ['id', 'value']
df_long.head(5)

,id,value
2016-07-01,A,532
2016-07-01,B,3314
2016-07-01,C,1136
2016-07-02,A,798
2016-07-02,B,2461


In [6]:
# ワイドフォーマットに戻す
df_wide = df_long.pivot(index=None, columns='id', values='value')
# 元通りになっている
df_wide.iloc[:5,]

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711
2016-07-04,937,5451,1977
2016-07-05,881,4729,1975


# 3.10.4 ラグ特徴量

>上記のケースcの場合のデータは過去の時点での値をそのまま特徴量にする、ラグ特徴量が非常に効果的。\
>WHY?->予測対象の値に対して自身の過去の値、特に直近の値が大きな影響を及ぼすため


## 単純なラグ特徴量

In [8]:
# xはワイドフォーマットのデータフレーム
# インデックスが日付などの時間、列がユーザや店舗などで、値が売上などの注目する変数を表すものとする
# 1期前のlagを取得
x=df_wide.copy()
x_lag1 = x.shift(1)

# 7期前のlagを取得
x_lag7 = x.shift(7)

In [9]:
x

id,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711
2016-07-04,937,5451,1977
2016-07-05,881,4729,1975
...,...,...,...
2016-12-27,840,4573,1850
2016-12-28,943,4511,1764
2016-12-29,978,4599,1787
2016-12-30,907,4243,2069


In [11]:
x_lag7

id,A,B,C
2016-07-01,NaN,NaN,NaN
2016-07-02,NaN,NaN,NaN
2016-07-03,NaN,NaN,NaN
2016-07-04,NaN,NaN,NaN
2016-07-05,NaN,NaN,NaN
...,...,...,...
2016-12-27,890.0000,3935.0000,2085.0000
2016-12-28,754.0000,4846.0000,2226.0000
2016-12-29,992.0000,4949.0000,2181.0000
2016-12-30,854.0000,4619.0000,2035.0000


## 移動平均やその他のラグ特徴量

周期的な動きがある場合、その周期に従って移動平均をとると、その影響を打ち消すことができる。\
例）日次のデータに対して7期の移動平均をとると、各曜日が必ず一度ずつ集計に含まれるため、曜日による変動の影響が打ち消され、その週の全体的な傾向をとらえることができる。\
pandasの**rolling関数とmeanなどの要約関数**を組み合わせて、以下のように移動平均を計算する

In [12]:
# 1期前から3期間の"移動平均"を算出
# 一番古い3日分に関しては過去3日分(1期前〜3期前)のデータがないためNull値となる
x_avg3 = x.shift(1).rolling(window=3).mean()

# 1期前から7期間の"最大値"を算出
x_max7 = x.shift(1).rolling(window=7).max()

In [14]:
x_lag1

id,A,B,C
2016-07-01,NaN,NaN,NaN
2016-07-02,532.0000,3314.0000,1136.0000
2016-07-03,798.0000,2461.0000,1188.0000
2016-07-04,823.0000,3522.0000,1711.0000
2016-07-05,937.0000,5451.0000,1977.0000
...,...,...,...
2016-12-27,840.0000,4576.0000,1954.0000
2016-12-28,840.0000,4573.0000,1850.0000
2016-12-29,943.0000,4511.0000,1764.0000
2016-12-30,978.0000,4599.0000,1787.0000


In [13]:
x_avg3

id,A,B,C
2016-07-01,NaN,NaN,NaN
2016-07-02,NaN,NaN,NaN
2016-07-03,NaN,NaN,NaN
2016-07-04,717.6667,3099.0000,1345.0000
2016-07-05,852.6667,3811.3333,1625.3333
...,...,...,...
2016-12-27,817.3333,4956.0000,1976.6667
2016-12-28,797.3333,4726.0000,1878.6667
2016-12-29,874.3333,4553.3333,1856.0000
2016-12-30,920.3333,4561.0000,1800.3333


やや処理が複雑になるが、周期に従って間隔を空けてデータを集計するアイデアもある

In [15]:
# 7期前, 14期前, 21期前, 28期前の値の平均
x_e7_avg = (x.shift(7) + x.shift(14) + x.shift(21) + x.shift(28)) / 4.0

>どのくらい過去のデータまでを集計するべきか

どれくらい過去のデータまでを集計して平均をとるべきかは、データの性質に依存するため都度見極めが必要。\
古い情報を集計し過ぎると直近の状況を表さない平均となってしまう場合もあったり、長期間であまり傾向が変わらないようなデータであれば、長い期間で集計した方が有利な場合もある。\
より直近の情報に重みを付ける加重移動平均や指数平滑平均を適用する方法もある。

> 自身の系列だけでなく、他の系列も集計してラグをとる

その店舗の過去の売上だけでなく、その店舗がある地域の売上の平均といったように、グルーピングして集計したものについてラグ特徴量を作成することもできる。\
「3.9.4 集計する単位を変える」と同様に、さまざまな集計単位や条件を考えることができる。

> 目的変数以外のラグをとる

ラグ特徴量として、目的変数以外の変数のラグをとることもできる。\
例えば、売上と一緒にその日の天候も与えられることがある（当日の天候だけでなく、前日の天候も当日の行動に影響する可能性があるので特徴量に入れてみるなど）。

> リード特徴量

ラグ特徴量とは逆に、1期後の値などの将来の値を特徴量とすることもできる。\
これをリード特徴量と言う。例えば翌日の天候(の予報)やキャンペーンが当日の行動に影響することがあるためリード特徴量として組み込むことができる。


In [16]:
# 1期先の値を取得
x_lead1 = x.shift(-1)
x_lead1.head(5)

id,A,B,C
2016-07-01,798.0000,2461.0000,1188.0000
2016-07-02,823.0000,3522.0000,1711.0000
2016-07-03,937.0000,5451.0000,1977.0000
2016-07-04,881.0000,4729.0000,1975.0000
2016-07-05,931.0000,4694.0000,1937.0000


# 3.10.5 時点と紐づいた特徴量を作る

データを予測する時点より過去の情報のみを使う制約を守ったまま学習・予測を行うために、時点と紐付いた特徴量を作り、時点をキーとして学習データと結合するという方法がある

<b>主なステップ</b>
1. 特徴量を作る元となるデータを、集計するなどして時点ごとに紐付いた変数とする
2. 必要に応じて、累積和や移動平均などをとる、他の変数との差や割合をとるといった処理を行う
3. 時点をキーとして学習データと結合する

<b>例</b>

不定期にイベントを開催している場合に、イベントの新鮮味や継続して人気があるイベントかを反映するために、その日が何回目の出現かを特徴量とすることを考える。\
より具体的に、セールというイベントの累積出現回数を表す特徴量を作成し、それを学習データと結合するには、以下のような流れで行う。

1. 各日付について、セールが開催されていたら1、そうでなければ0とする 
2. 累積和をとることで、各日付でのセールの累積出現回数を求める
3. 日付をキーとして学習データと結合する

*ある時点から過去1か月間の出現回数であったり、すべてのイベントに対するそのイベントの比率を特徴量とすることもできる

In [30]:
# train_xは学習データでユーザーID, 日付を例として持つDataFrameとする
# event_historyは、過去に開催したイベントの情報で、日付、イベントを列として持つDataFrameとする
train_x = read_csv('time_series_train')
event_history = read_csv('time_series_events')
## dateをdatetimeデータに変換
train_x['date'] = pd.to_datetime(train_x['date'])
event_history['date'] = pd.to_datetime(event_history['date'])

Load: /Users/satoshiido/Documents/coding_general/signate/LT#1/inputs/time_series_train.csv
Load: /Users/satoshiido/Documents/coding_general/signate/LT#1/inputs/time_series_events.csv


In [31]:
train_x.head()

,user_id,date,target
0,1,2018-01-01,1
1,1,2018-01-02,1
2,1,2018-01-03,1
3,1,2018-01-04,1
4,1,2018-01-05,0


In [32]:
event_history.iloc[:10, :]

,date,event
0,2018-01-03,sale
1,2018-01-03,conpon
2,2018-01-04,points
3,2018-01-05,points
4,2018-05-03,sale
5,2018-05-04,sale
6,2018-05-05,sale
7,2018-05-06,points
8,2018-05-07,points
9,2018-05-08,points


In [33]:
# occurrencesは日付、セールが開催されたか否かを列として持つDataFrameとなる
dates = np.sort(train_x['date'].unique())
## dataframe型に変換させる
occurrences = pd.DataFrame(dates, columns=['date'])
## event_historyでsaleとなっているrowを取得してsale_historyとする
sale_history = event_history[event_history['event']=='sale']
## sale_historyのdateに含まれているdateと同じdateの場合occurrencesのsaleという新しいカラムをTrueとする
occurrences['sale'] = occurrences['date'].isin(sale_history['date'])

In [34]:
# 日付をキーとして学習データと結合する
train_x = pd.merge(train_x, occurrences, on='date', how='left')
train_x.head()

,user_id,date,target,sale
0,1,2018-01-01,1,False
1,1,2018-01-02,1,False
2,1,2018-01-03,1,True
3,1,2018-01-04,1,False
4,1,2018-01-05,0,False


# 3.10.6 予測に使えるデータの期間

<b>特徴量を作るときに使える過去の期間</b>

テストデータのレコードの時点ごとに、使える過去の期間が異なる場合、使えるデータの期間についてもう一段階注意が必要である。

* 時系列データを扱う分析コンペでは、学習データとテストデータは時間的に分割され、あるテストデータの期間の予測をまとめて提出するケースが多い。\
この場合、テストデータの特徴量を作るときに使えるデータの期間に制限が生じる。

* テストデータには目的変数の値が含まれていないため、目的変数に関しては分割時点より過去の値しか参照できない。\
テストデータが1か月間ある場合、本書図3.29のように分割時点直後のデータでは1日前の目的変数を参照できるが、1か月後のデータでは1か月前の目的変数しか参照できない。\
その条件を学習やバリデーションにおいても踏襲しておかないと、特徴量の性質がテストデータと異なってしまう。\
その結果、テストデータに対する汎化性能が落ちたり、バリデーションで不当に高い制度が出てしまう不具合が生じてしまう。


* この場合のアプローチの1つは、テストデータの期間が分割時点から何期先であるかによって、個別にモデルを作るという方法が挙げられる。\
つまり、分割時点の翌日のデータでは1日前のラグ変数が使えるため、その前提で特徴量を作り、モデルの学習・バリデーション・テストデータへの予測をする。\
一方で、分割時点の1か月後のデータでは1か月前のラグ変数しか使えない前提で同様に特徴量やモデルを作り、予測する。\
（＊Kaggleの「Corporación Favorita Grocery Sales Forecasting」や「Recruit Restaurant Visitor Forecasting」は、前者は各商品の販売数、後者は飲食店の来客数を数期先に渡って予測するタスクで、上記のようにテストデータのあとの方の期間になるほど目的変数のラグが古いものしか使えない状況だった。上位のアプローチとして、上述した個別にモデルを作る方法が用いられていた。）

* カレンダー情報などは予測時点で既知の情報なので良いが、そうでない将来の情報を使ったモデルは実用上は適切ではないとのこと。分析コンペにおいては、テストデータが一括で与えられるため予測時点より将来の情報が利用できることもあります。それによって精度が上がるのであれば、リード特徴量などを作り予測に役立てる必要がある。
（＊なお、分析コンペによってはルールで禁止されていたり、不可能になっていることがある。SIGNATEの「Jリーグの観客動員数予測」では、予測対象の日以前に確定している情報のみを用いて予測を行うこと、というルールが明示されていた。 また、Kaggleの「Two Sigma Financial Modeling Challenge」 では、 参加者は Kaggle Kernelを通じてプログラムコードを提出し、学習・予測をサーバ側で行うため、将来の情報を使えない環境で予測を行う制約を実現していた。）